In [1]:
# 라이브러리 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By



## 네이버 View 검색을 활용한 블로그 검색
- 스크롤 활용
- 월 별로 1000건 제한 걸어둔 네이버

In [2]:
# 크롤링 시작과 끝 날짜 (달 별로 크롤링)
first_days = pd.date_range('2021/01/01','2022/10/31', freq='MS')#각 달의 첫 날
last_days = pd.date_range('2021/01/01','2022/10/31', freq='M') # 각 달의 마지막 날

keyword_list =['레스토랑간편식','가정간편식'] #검색 키워드 추가

In [3]:
#크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource
# 현재 컴큐터 크롬 드라이버 위치
chorme_path = './chromedriver.exe' #본인의 크롬 드라이버 위치 입력

In [4]:
#블로그 주소 수집
blog_url_list = []

#네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    # 자동화된 크롬 창 실행
    driver = webdriver.Chrome(executable_path=chorme_path, options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 990 #한번에 1000개밖에 못긁어옴
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭
    
    #날짜 지정
    for k in range(len(first_days)):
        try:
            #시작과 끝의 년,월,일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()
            
            #스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30)-1 #한 페이지에 기본으로 30개 글이 나옴
            
            #스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                time.sleep(random.uniform(1,1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                last_height = new_height
                
            soup = bs(driver.page_source,'lxml')
            blog_url = soup.find_all('a', class_='api_txt_lines total_tit')
            
            #url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) #주소 수집
            
            print('{0} 키워드 {1}년 {2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url),'개 블로그URL수집')
            
            driver.execute_script('window.scrollTo(0,0)') #수집 완료 후 스크롤 더 내려가기 방지
            
        except:
            pass

레스토랑간편식 키워드 2021년 1월 390 개 블로그URL수집
레스토랑간편식 키워드 2021년 2월 278 개 블로그URL수집
레스토랑간편식 키워드 2021년 3월 300 개 블로그URL수집
레스토랑간편식 키워드 2021년 4월 237 개 블로그URL수집
레스토랑간편식 키워드 2021년 5월 241 개 블로그URL수집
레스토랑간편식 키워드 2021년 6월 228 개 블로그URL수집
레스토랑간편식 키워드 2021년 7월 286 개 블로그URL수집
레스토랑간편식 키워드 2021년 8월 600 개 블로그URL수집
레스토랑간편식 키워드 2021년 9월 391 개 블로그URL수집
레스토랑간편식 키워드 2021년 10월 240 개 블로그URL수집
레스토랑간편식 키워드 2021년 11월 570 개 블로그URL수집
레스토랑간편식 키워드 2021년 12월 570 개 블로그URL수집
레스토랑간편식 키워드 2022년 1월 330 개 블로그URL수집
레스토랑간편식 키워드 2022년 2월 245 개 블로그URL수집
레스토랑간편식 키워드 2022년 3월 390 개 블로그URL수집
레스토랑간편식 키워드 2022년 4월 300 개 블로그URL수집
레스토랑간편식 키워드 2022년 5월 283 개 블로그URL수집
레스토랑간편식 키워드 2022년 6월 480 개 블로그URL수집
레스토랑간편식 키워드 2022년 7월 360 개 블로그URL수집
레스토랑간편식 키워드 2022년 8월 310 개 블로그URL수집
레스토랑간편식 키워드 2022년 9월 300 개 블로그URL수집
레스토랑간편식 키워드 2022년 10월 292 개 블로그URL수집
가정간편식 키워드 2021년 1월 990 개 블로그URL수집
가정간편식 키워드 2021년 2월 990 개 블로그URL수집
가정간편식 키워드 2021년 3월 990 개 블로그URL수집
가정간편식 키워드 2021년 4월 150 개 블로그URL수집
가정간편식 키워드 2021년 5월 990 개 블로그URL수집
가정간편식 키워드 2021년 6월 990 개 블로그URL수집


In [5]:
len(blog_url_list)

24841

In [6]:
#수집 후 URL list 중복 확인
real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

23080

In [7]:
#파일저장(블로그 주소)
k = pd.DataFrame(blog_url_list)
k.to_csv('url_list_밀키트.csv', encoding='utf-8-sig')

In [9]:
#블로그 크롤링
url = pd.read_csv('url_list_밀키트.csv',encoding='utf-8-sig')
blog_url_list = list(url['0'])

In [10]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_17296\769458119.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


네이버 블로그 아닌 url:  https://tokkisaru.tistory.com/entry/%EA%BF%89%EB%8B%B9-%EC%BF%A0%EC%8B%9C%EB%A7%88%EC%82%AC-%EA%B0%99%EC%9D%80-SNS%EB%A7%9B%EC%A7%91%EC%9D%84-%EC%9D%B4%EC%A0%9C-CU%EC%97%90%EC%84%9C-%EB%A7%8C%EB%82%A0-%EC%88%98-%EC%9E%88%EB%8B%A4
10개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://bhu2714.tistory.com/151
20개 블로그 크롤링 완료
30개 블로그 크롤링 완료
40개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://sooyu.tistory.com/524
50개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://monkeydaylife.tistory.com/entry/%EB%8F%88%EC%8A%A4%ED%8C%8C%EC%9D%B4%ED%81%AC-%EC%96%91%EB%85%90-%EC%AA%BD%EA%B0%88%EB%B9%84-%EA%B5%AC%EB%A7%A4-%ED%9B%84%EA%B8%B0-with-%EC%AA%BD%EA%B0%88%EB%B9%84%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B3%A0%EC%88%98-%ED%81%AC%EB%A6%BC-%ED%8C%8C%EC%8A%A4%ED%83%80
네이버 블로그 아닌 url:  https://byline.network/2021/05/12-134/
네이버 블로그 아닌 url:  https://blueplastic.tistory.com/6620
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://1conomynews.tistory.com/2251
80개 블로그 크롤링 완료
90개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://enterfn.t

In [11]:
len(time_list)

22428

In [12]:
data = {'time':time_list,
        'review':review_list,
        'like':like_list,
        'url':url_list,
        'comment': comment_list}

df = pd.DataFrame(data)

print('크롤링 소요시간', time.time()-start)
print('총 {}개 블로그 크롤링 완료!'.format(count))
df.head()

크롤링 소요시간 146900.92394256592
총 22437개 블로그 크롤링 완료!


,time,review,like,url,comment
0,2022. 6. 29. 23:50,뿜뿜 생활리뷰 집밥 생선구이는 헷구어로 고등어 삼치 연어 다 ...,[],https://blog.naver.com/role_p/222794171031,[]
1,2021. 9. 15. 22:31,iNDONESIAnEWS 현대 LG 인니 배터리 셀공장 착공...,4,https://blog.naver.com/esuccess/222507097150,[]
2,2021. 5. 29. 11:10,신개념정육점 썰다(3) 유영채영애비 ・ ...,2,https://blog.naver.com/woojinlov2/222373226012,[]
3,2021. 5. 28. 10:50,상품리뷰 양곱창전골 밀키트로 만들어먹기 쉽고간편한 한끼 ...,[],https://blog.naver.com/tim0510/222371400206,[]
4,2022. 5. 9. 9:43,김용현 국힘 구리시의원 예비후보 푸드테크밸리와 한강변 도시개발사업 관련 질문 답변...,3,https://blog.naver.com/2580press/222725695247,[]


In [13]:
#csv저장
df.to_csv('밀키트.csv', encoding='utf-8-sig')